<a href="https://colab.research.google.com/github/crazat/crazat.githurb.io/blob/main/HyperParameter_Tuning_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 타이타닉 생존자 데이터셋 다운로드 받아 구글드라이브에 올리기
    - 학습세트: https://drive.google.com/file/d/17DUdpAGVIPJ5_Cghku38YG5ugN6-SiNX/view?usp=sharing
    - 평가세트: https://drive.google.com/file/d/1ZhrzzI81mN_rFWXw68eO2a2NQ8vg8IlZ/view?usp=sharing
    - 평가세트 정답: https://drive.google.com/file/d/1kBJwGeAI1y81-gL8AAVVs4ealPHTcNNS/view?usp=sharing

- 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 데이터 경로 변수

In [ ]:
DATA_PATH = "/content/drive/MyDrive/머신러닝/data/"
DATA_PATH

'/content/drive/MyDrive/머신러닝/data/'

- 시드값

In [ ]:
SEED = 42

- 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv(f"{DATA_PATH}titanic_train.csv") # 학습데이터
test = pd.read_csv(f"{DATA_PATH}titanic_test.csv") # 테스트 데이터
test_target = pd.read_csv(f"{DATA_PATH}titanic_test_target.csv") # 테스트데이터 정답값
train.shape , test.shape , test_target.shape

((916, 12), (393, 11), (393, 2))

In [ ]:
train.head()

,passengerid,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,494,0,1,"Artagaveytia, Mr. Ramon",male,71.0,0,0,PC 17609,49.5042,NaN,C
1,462,0,3,"Morley, Mr. William",male,34.0,0,0,364506,8.0500,NaN,S
2,1286,0,3,"Kink-Heilmann, Mr. Anton",male,29.0,3,1,315153,22.0250,NaN,S
3,1130,1,2,"Hiltunen, Miss. Marta",female,18.0,1,1,250650,13.0000,NaN,S
4,461,1,1,"Anderson, Mr. Harry",male,48.0,0,0,19952,26.5500,E12,S


- 결측치 확인하기

In [ ]:
train.isnull().sum()

,0
passengerid,0
survived,0
pclass,0
name,0
gender,0
age,180
sibsp,0
parch,0
ticket,0
fare,0


In [ ]:
test.isnull().sum()

,0
passengerid,0
pclass,0
name,0
gender,0
age,83
sibsp,0
parch,0
ticket,0
fare,1
cabin,308


- 학습 데이터에서 얻은 통계량을 이용하여 결측치를 채워야한다.

In [ ]:
age_mean = train["age"].mean() # 평균
fare_median = train["fare"].median() # 중앙값
cabin_unk = "UNK" # 새로운 범주
embarked_mode = train["embarked"].mode()[0] # 최빈값
age_mean , fare_median ,cabin_unk , embarked_mode

(29.904891304347824, 14.5, 'UNK', 'S')

- 학습데이터 결측치 처리

In [ ]:
train["age"] = train["age"].fillna(age_mean)
train["cabin"] = train["cabin"].fillna(cabin_unk)

- 테스트데이터 결측치 처리

In [ ]:
test["age"] = test["age"].fillna(age_mean)
test["fare"] = test["fare"].fillna(fare_median)
test["cabin"] = test["cabin"].fillna(cabin_unk)
test["embarked"] = test["embarked"].fillna(embarked_mode)

In [ ]:
train.isnull().sum().sum() , test.isnull().sum().sum()

(0, 0)

- 특성으로 사용할 변수 추가하기

In [ ]:
cols = ["age","sibsp","parch","fare","pclass","gender","embarked"]
train_ft = train[cols].copy()
test_ft = test[cols].copy() # 테스트 데이터
train_ft.shape, test_ft.shape

((916, 7), (393, 7))

- 범주형 변수 원핫인코딩하여 특성으로 추가하기

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cols = ['gender','embarked']
enc = OneHotEncoder(handle_unknown = 'ignore') # 모르는 범주가 있을 경우 무시
enc.fit(train[cols])

OneHotEncoder(handle_unknown='ignore')

In [ ]:
# 학습 데이터
tmp = pd.DataFrame(
    enc.transform(train_ft[cols]).toarray(), # ndarray
    columns = enc.get_feature_names_out() # 컬럼명
)
train_ft = pd.concat([train_ft,tmp],axis=1)
train_ft.head()

,age,sibsp,parch,fare,pclass,gender,embarked,gender_female,gender_male,embarked_C,embarked_Q,embarked_S
0,71.0,0,0,49.5042,1,male,C,0.0,1.0,1.0,0.0,0.0
1,34.0,0,0,8.0500,3,male,S,0.0,1.0,0.0,0.0,1.0
2,29.0,3,1,22.0250,3,male,S,0.0,1.0,0.0,0.0,1.0
3,18.0,1,1,13.0000,2,female,S,1.0,0.0,0.0,0.0,1.0
4,48.0,0,0,26.5500,1,male,S,0.0,1.0,0.0,0.0,1.0


In [ ]:
# 테스트 데이터
tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft,tmp],axis=1)
test_ft.head()

,age,sibsp,parch,fare,pclass,gender,embarked,gender_female,gender_male,embarked_C,embarked_Q,embarked_S
0,62.000000,0,0,26.5500,1,male,S,0.0,1.0,0.0,0.0,1.0
1,28.000000,0,0,47.1000,1,male,S,0.0,1.0,0.0,0.0,1.0
2,24.000000,0,0,9.5000,3,male,S,0.0,1.0,0.0,0.0,1.0
3,29.904891,0,0,7.7333,3,female,Q,1.0,0.0,0.0,1.0,0.0
4,18.500000,0,0,7.2833,3,female,Q,1.0,0.0,0.0,1.0,0.0


- 문자열 데이터 제거

In [ ]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)

- Min-Max Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_ft)

MinMaxScaler()

In [ ]:
train_ft[train_ft.columns] = scaler.transform(train_ft) # 학습 데이터
train_ft.head()

,age,sibsp,parch,fare,pclass,gender_female,gender_male,embarked_C,embarked_Q,embarked_S
0,0.887260,0.000,0.000000,0.096626,0.0,0.0,1.0,1.0,0.0,0.0
1,0.423776,0.000,0.000000,0.015713,1.0,0.0,1.0,0.0,0.0,1.0
2,0.361142,0.375,0.111111,0.042990,1.0,0.0,1.0,0.0,0.0,1.0
3,0.223350,0.125,0.111111,0.025374,0.5,1.0,0.0,0.0,0.0,1.0
4,0.599148,0.000,0.000000,0.051822,0.0,0.0,1.0,0.0,0.0,1.0


In [ ]:
test_ft[test_ft.columns] = scaler.transform(test_ft) # 테스트 데이터
test_ft.head()

,age,sibsp,parch,fare,pclass,gender_female,gender_male,embarked_C,embarked_Q,embarked_S
0,0.774521,0.0,0.0,0.051822,0.0,0.0,1.0,0.0,0.0,1.0
1,0.348616,0.0,0.0,0.091933,0.0,0.0,1.0,0.0,0.0,1.0
2,0.298509,0.0,0.0,0.018543,1.0,0.0,1.0,0.0,0.0,1.0
3,0.372478,0.0,0.0,0.015094,1.0,1.0,0.0,0.0,1.0,0.0
4,0.229613,0.0,0.0,0.014216,1.0,1.0,0.0,0.0,1.0,0.0


- 정답 데이터

In [ ]:
target = train["survived"]
target

,survived
0,0
1,0
2,0
3,1
4,1
...,...
911,1
912,0
913,0
914,0


# Model Tuning (Hyperparameter Optimization)
- 학습을 수행하기 전에 설정해야 하는 값인 hyperparameter의 최적값을 탐색

## Grid Search
- 하이퍼파라미터 별로 다양한 값들을 지정하여 가능한 모든 조합에 대해 성능 결과를 측정한 후 가장 높은 성능을 발휘했던 하이퍼파라미터 조합을 최적값으로 선정하는 방법
- 탐색하고자 하는 하이퍼파라미터의 개수를 여러 종류로 가져갈수록 가능한 모든 조합을 탐색하기 때문에 전체 탐색 시간이 기하급수적으로 증가한다는 단점이 있음
- GridSearchCV 클래스
    - estimator(첫번째 파라미터)
        - 모델 객체
    - param_grid(두번째 파라미터)
        - 탐색 구간을 딕셔너리로 전달
        - key는 하이퍼파라미터명
        - value는 탐색구간(iterable 객체)
    - scoring
        - 평가지표(문자열로 전달)
        - https://scikit-learn.org/stable/modules/model_evaluation.html
    - cv
        - cv 객체 또는 정수

In [ ]:
# 튜닝할때 일반적으로 교차검증하여 튜닝한다.
from sklearn.model_selection import KFold
cv = KFold(n_splits=5,shuffle=True, random_state=SEED)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
hp = {
    "n_estimators" : range(100,121,10) , # 트리 수
    "criterion" : ["gini","entropy"] , # 순수도 척도
    "max_depth" : range(5,11,5) , # 트리 최대 깊이
    "min_samples_split" : range(2,5,2), # 노드를 분할하는 데 필요한 최소 샘플 수
    "max_features" : ["sqrt","log2"], # 트리가 사용할 피쳐 수
}

model = RandomForestClassifier(random_state=SEED)
grid_search = GridSearchCV(
    model, # 모델 객체
    hp, # 하이퍼 파라미터의 목록
    cv=cv,
    scoring="roc_auc",
    n_jobs = -1
)

grid_search.fit(train_ft, target)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(5, 11, 5),
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_split': range(2, 5, 2),
                         'n_estimators': range(100, 121, 10)},
             scoring='roc_auc')

- 교차검증 평균 점수

In [ ]:
grid_search.best_score_

0.9144457173786703

- 최적의 하이퍼파라미터

In [ ]:
grid_search.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_split': 4,
 'n_estimators': 100}

- 최적의 하이퍼파라미터를 모델에 적용하여 학습데이터 전체를 다시 학습했기 때문에 예측도 가능

In [ ]:
pred = grid_search.predict_proba(test_ft)[:,1]
pred[:5]

array([0.17601201, 0.19401594, 0.03364182, 0.92101312, 0.77181654])

- 최적의 하이퍼파라미터 모델에 적용해보기

In [ ]:
from sklearn.model_selection import cross_val_score
model = RandomForestClassifier(random_state=SEED, **grid_search.best_params_) # 튜닝된 하이퍼파라미터  적용
scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='roc_auc',n_jobs = -1)
print(scores) # 폴드별 검증점수 리스트
np.mean(scores) # cv 평균 점수

[0.92550125 0.95623605 0.90132616 0.87727496 0.91189016]


0.9144457173786703

In [ ]:
model.fit(train_ft,target)
pred = grid_search.predict_proba(test_ft)[:,1]
pred[:5]

array([0.17601201, 0.19401594, 0.03364182, 0.92101312, 0.77181654])

## Random Search
- 하이퍼파라미터 별로 탐색 구간을 지정하여 가능한 모든 조합 중에 지정한 횟수만큼 하이퍼파라미터 조합을 랜덤 샘플링하여 성능 결과를 측정한 후 가장 높은 성능을 발휘했던 하이퍼파라미터 조합을 최적값으로 선정하는 방법
- Random Search는 Grid Search에 비해 불필요한 반복 수행 횟수를 대폭 줄이면서 동시에 정해진 간격 사이에 위치한 값들에 대해서도 확률적으로 탐색이 가능하므로 최적 하이퍼파라미터 값을 더 빨리 찾을 수 있는 것으로 알려져 있음
- RandomizedSearchCV
    - estimator(첫번째 파라미터)
        - 모델 객체
    - param_distributions(두번째 파라미터)
        - 탐색 구간을 딕셔너리로 전달
        - key는 하이퍼파라미터명
        - value는 탐색구간(iterable 객체)
    - n_iter
        - 탐색 횟수
    - scoring
        - 평가지표(문자열로 전달)
    - cv
        - cv 객체 또는 정수
    - random_state
        - 시드값

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
hp = {
    "n_estimators" : range(100,201,2) , # 트리 수
    "criterion" : ["gini","entropy"] , # 순수도 척도
    "max_depth" : range(5,51,2) , # 트리 최대 깊이
    "min_samples_split" : range(2,21,1), # 노드를 분할하는 데 필요한 최소 샘플 수
    "max_features" : ["sqrt","log2",None], # 트리가 사용할 피쳐 수
}

model = RandomForestClassifier(random_state=SEED)
rand_search = RandomizedSearchCV(
    model, # 모델 객체
    hp, # 하이퍼 파라미터의 목록
    n_iter = 20,
    cv=cv,
    scoring="roc_auc",
    n_jobs = -1,
    random_state=SEED
)

rand_search.fit(train_ft, target)

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=RandomForestClassifier(random_state=42), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': range(5, 51, 2),
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_split': range(2, 21),
                                        'n_estimators': range(100, 201, 2)},
                   random_state=42, scoring='roc_auc')

- 교차검증 평균 점수

In [ ]:
rand_search.best_score_

0.9150661196469378

- 최적의 하이퍼파라미터

In [ ]:
rand_search.best_params_

{'n_estimators': 158,
 'min_samples_split': 14,
 'max_features': 'log2',
 'max_depth': 29,
 'criterion': 'entropy'}

- 예측도 가능

In [ ]:
pred = rand_search.predict_proba(test_ft)[:,1]
pred[:5]

array([0.25382235, 0.19881591, 0.05048433, 0.89957257, 0.7804184 ])

## optuna
- 베이지안 최적화 방식의 하이퍼파라미터 튜닝을 적용할 수 있는 파이썬 라이브러리
- https://optuna.readthedocs.io/en/stable/index.html
- 주피터 노트북 환경에서 설치 방법
```python
!pip install optuna
```

- 설치

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.0 MB/s eta 0:00:00


### optuna를 이용한 튜닝 방법
1. 목적함수를 만든다
    - optuna 에서는 목적함수에 trial 객체를 전달
    - trial 객체의 suggest 메소드를 이용하여 하이퍼파라미터를 세팅
        - trial 객체의 suggest 메소드가 사전지식을 반영하여 하이퍼파라미터 제안
    - 모델 학습및 검증평가 결과를 반환
2. 대체 모델 역할을 하는 Sampler 객체를 생성
3. 전체적인 튜닝과정을 진행하는 스터디 객체를 생성하여 optimize 메소드로 튜닝 시작



In [ ]:
import optuna

- 목적함수 만들기


In [ ]:
# optuna.trial.Trial.suggest_int
# optuna.trial.Trial.suggest_categorical
# optuna.trial.Trial.suggest_float

In [ ]:
def objective(trial): # trial 객체를 받음

    # 하이퍼 파라미터 탐색 범위 정의
    hp = {
        'n_estimators' : trial.suggest_int('n_estimators',80,250),
        "criterion" : trial.suggest_categorical("criterion",["gini","entropy"]),
        'max_depth' : trial.suggest_int('max_depth',5,20),
        'min_samples_split' : trial.suggest_int('min_samples_split',2,20),
        "max_features" : trial.suggest_categorical("max_features",["sqrt","log2",None]),
    }

    # cv 객체 및 모델 객체 생성
    cv = KFold(n_splits=5,shuffle=True, random_state=SEED)
    model = RandomForestClassifier(random_state=SEED,**hp)

    # 검증
    scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='roc_auc')
    return np.mean(scores) # 평균

In [ ]:
sampler = optuna.samplers.TPESampler(seed=SEED) # Tree-Structured Parzen Estimator 로 많이 사용되는 대체 모델

# 스터디 객체 생성
study = optuna.create_study(
    direction="maximize", # 평가지표에 따라 maximize, minimize 선택
    sampler=sampler,  # 샘플러 객체 전달
)

# 튜닝 시작
study.optimize(objective, n_trials=50) # 첫번째 인수에 목적함수의 주소를 전달하고, n_trials 은 시도횟수
print("Best Score:", study.best_value) # 최적의 하이퍼파라미터 검증점수
print("Best trial:", study.best_trial.params) # 최적의 하이퍼파라미터 조합

[I 2025-02-26 07:16:57,176] A new study created in memory with name: no-name-280f3183-566f-4bc1-94c5-f706f688d38f
[I 2025-02-26 07:17:03,148] Trial 0 finished with value: 0.9021876576369428 and parameters: {'n_estimators': 144, 'criterion': 'gini', 'max_depth': 14, 'min_samples_split': 4, 'max_features': None}. Best is trial 0 with value: 0.9021876576369428.
[I 2025-02-26 07:17:07,138] Trial 1 finished with value: 0.9135523040367891 and parameters: {'n_estimators': 182, 'criterion': 'gini', 'max_depth': 20, 'min_samples_split': 17, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.9135523040367891.
[I 2025-02-26 07:17:11,529] Trial 2 finished with value: 0.9106241540175015 and parameters: {'n_estimators': 132, 'criterion': 'gini', 'max_depth': 9, 'min_samples_split': 13, 'max_features': None}. Best is trial 1 with value: 0.9135523040367891.
[I 2025-02-26 07:17:17,951] Trial 3 finished with value: 0.9134495345891344 and parameters: {'n_estimators': 157, 'criterion': 'gini', 'max_de

Best Score: 0.9162408095358433
Best trial: {'n_estimators': 102, 'criterion': 'entropy', 'max_depth': 13, 'min_samples_split': 20, 'max_features': 'sqrt'}


- 클래스로 만드는 예시

In [ ]:
# train_ft, target , SEED

In [ ]:
class Objective:
    def __init__(self,x_train,y_train,seed):
        self.x_train = x_train # Features
        self.y_train = y_train # target
        self.seed = seed # 시드값
        self.cv = KFold(n_splits=5,shuffle=True, random_state=seed) # cv 객체
    def __call__(self,trial): # 객체를 함수처럼 사용할수 있게 해주는 매직 메소드
        hp = {
            'n_estimators' : trial.suggest_int('n_estimators',80,250),
            "criterion" : trial.suggest_categorical("criterion",["gini","entropy"]),
            'max_depth' : trial.suggest_int('max_depth',5,20),
            'min_samples_split' : trial.suggest_int('min_samples_split',2,20),
            "max_features" : trial.suggest_categorical("max_features",["sqrt","log2",None]),
        }

        model = RandomForestClassifier(random_state=self.seed,**hp)
        scores = cross_val_score(model,self.x_train,self.y_train,cv = self.cv ,scoring='roc_auc')
        return np.mean(scores)

In [ ]:
sampler = optuna.samplers.TPESampler(seed=SEED)

study = optuna.create_study(
    direction="maximize",
    sampler=sampler,
)
objective = Objective(train_ft,target,SEED) # 목적함수 역할을 하는 Objective 클래스 객체 생성
study.optimize(objective, n_trials=50)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2025-02-26 07:18:40,721] A new study created in memory with name: no-name-9ad65752-799d-4da8-8046-548f85ccfc4b
[I 2025-02-26 07:18:45,264] Trial 0 finished with value: 0.9021876576369428 and parameters: {'n_estimators': 144, 'criterion': 'gini', 'max_depth': 14, 'min_samples_split': 4, 'max_features': None}. Best is trial 0 with value: 0.9021876576369428.
[I 2025-02-26 07:18:49,578] Trial 1 finished with value: 0.9135523040367891 and parameters: {'n_estimators': 182, 'criterion': 'gini', 'max_depth': 20, 'min_samples_split': 17, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.9135523040367891.
[I 2025-02-26 07:18:52,961] Trial 2 finished with value: 0.9106241540175015 and parameters: {'n_estimators': 132, 'criterion': 'gini', 'max_depth': 9, 'min_samples_split': 13, 'max_features': None}. Best is trial 1 with value: 0.9135523040367891.
[I 2025-02-26 07:18:59,389] Trial 3 finished with value: 0.9134495345891344 and parameters: {'n_estimators': 157, 'criterion': 'gini', 'max_de

Best Score: 0.9162408095358433
Best trial: {'n_estimators': 102, 'criterion': 'entropy', 'max_depth': 13, 'min_samples_split': 20, 'max_features': 'sqrt'}


- 하이퍼파라미터 중요도

In [ ]:
optuna.visualization.plot_param_importances(study)

- 탐색 과정에서 모든 시도에 대한 history

In [ ]:
optuna.visualization.plot_optimization_history(study)

# 테스트 데이터 평가해보기

In [ ]:
study.best_trial.params

{'n_estimators': 102,
 'criterion': 'entropy',
 'max_depth': 13,
 'min_samples_split': 20,
 'max_features': 'sqrt'}

- 학습

In [ ]:
model = RandomForestClassifier(random_state=SEED, **study.best_trial.params) # 튜닝된 하이퍼파라미터  적용
model.fit(train_ft,target)

RandomForestClassifier(criterion='entropy', max_depth=13, min_samples_split=20,
                       n_estimators=102, random_state=42)

- 테스트셋 예측

In [ ]:
pred = model.predict_proba(test_ft)[:,1] # 예측
pred[:5]

array([0.27382469, 0.19367161, 0.04288819, 0.87531551, 0.83110202])

- 최종 평가

In [ ]:
from sklearn.metrics import roc_auc_score
y_test = test_target["survived"] # 테스트셋 y값
roc_auc_score(y_test,pred) # AUC 평가

0.8875068946497517